# markdown.obsidian.personal.note_type

> Functions for distinguishing types of [Obsidian.md](https://obsidian.md/) notes that I use.

In [2]:
#| default_exp markdown.obsidian.personal.note_type

In [3]:
#| export
from deprecated import deprecated
from enum import Enum, auto
import os
from pathlib import Path
from typing import Union
import warnings

from trove.helper import (
    file_existence_test
)
from trove.markdown.obsidian.vault import (
    note_path_by_name, note_name_unique, VaultNote, NoteDoesNotExistError
)

ModuleNotFoundError: No module named 'trove'

It should be convenient to be able to distinguish between different types of notes; thus far, I have made the following types of notes:

1. index notes, which list other notes, usually standard information notes and usually in the order that they are found in texts/papers.
2. "standard" information notes, which display a block of content or blocks or related content.
3. "module" information notes, which contain a block of information to be embedded into other notes for common use.
4. index of notation notes, which list notation used in a reference.
5. notation notes, which describe what a single notation or few notations denote.
6. premise notes, which relay commonly used notations/assumptions and which are embedded into 
7. common terms notes, which list out terms that have similar names that may or may not be related.
8. templates, which are templates for standard information notes and index of notation notes.
9. tags, which list out tags of similar concepts 
10. notes, which are notes of videos/lectures
11. mathematician notes, which serve as a hub for basic information about individual mathematicians
12. equation notes, which are used to embed and reference equations or, more generally, mathematical expressions. 
13. reference notes, which are usually embedded in "standard" information notes and list basic details about references/texts, including authors and where the reference is available online.
14. terms notes, which list which notes serve as links to which terms in a reference.

In [ ]:
#| export
# TODO: Make these methods private.
def _is_index_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be an index note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function only looks at the file name, so it may return
    incorrect results if the note is named incorrectly.
    """
    # file = file_existence_test(file, vault)
    return vault_note.name.startswith('_index')


def _is_index_of_notation_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be an index of notation note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function only looks at the file name, so it may return
    incorrect results if the note is named incorrectly.
    """
    return vault_note.name.startswith('_notation')


def _is_standard_information_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be a standard information note.

    This function looks for some headings to verify that the note
    is a standard information note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function may return incorrect results, especially
    false negatives.
    - This function returns True if a few tests pass, but will
    also print out the file name if not enough tests pass.
    """
    # file = file_existence_test(file, vault)
    if (_is_index_of_notation_note(vault_note)
        or _is_template_note(vault_note)):
        return False
    # TODO
    text = vault_note.text()
    tokens = ['# Meta', '## References',
              '## Citations and Footnotes']
    passes = [x in text for x in tokens]
    total_passes = sum(passes)
    if total_passes == 0:
        return False
    elif total_passes <= 2:
        warnings.warn('The following file seems to be a standard information'
                      f' note, but this is ambiguous: {vault_note.name}')
    return True


# TODO: delete
@deprecated
def _is_module_information_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be a module information note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function may return incorrect results.
    """
    # TODO figure out which files the card is embedded in
    return 


def _is_notation_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be a notation note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function looks at the file name, so it may return
    incorrect results if the note is named incorrectly.
    """
    if (_is_index_of_notation_note(vault_note)
            or _is_standard_information_note(vault_note)):
        return False
    return 'notation' in vault_note.name


def _is_premise_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be a premise note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function may return incorrect results.
    """
    # TODO figure out which files the card is embedded in
    # TODO test
    return 'premise' in vault_note.name


def _is_common_terms_note(vault_note: VaultNote) -> bool:
    """Returns `True` if the markdown file is determined to
    be a common terms note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function only looks at the file name, so it may return
    incorrect results if the note is named incorrectly.
    """
    return vault_note.name.startswith('_common_terms')


def _is_template_note(vault_note: VaultNote):
    """Returns `True` if the markdown file is determined to
    be a template note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function only looks at the file name, so it may return
    incorrect results if the note is named incorrectly.
    """
    return vault_note.name.startswith('_template')


# TODO: delete
@deprecated
def _is_tag_note(vault_note: VaultNote):
    return


def _is_notes_note(vault_note: VaultNote):
    # TODO
    """Returns `True` if the markdown file is determined to
    be a notes note.

    A notes note usually is self-written.
    
    """
    return


def _is_mathematician_note(vault_note: VaultNote):
    """Returns `True` if the markdown file is determined to
    be a mathematician note.

    **Parameters**
    - `vault_note` - VaultNote
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function only looks for headers `# Basic` and
    `# Notables`, so it may return incorrect results if the
    note is formatted incorrectly.
    """
    text = vault_note.text()
    tokens = ['# Basic', '# Notables']
    passes = [x in text for x in tokens]
    total_passes = sum(passes)
    return total_passes != 0


def _is_equation_note(vault_note: VaultNote):
    # TODO
    return


def _is_reference_note(vault_note: VaultNote):
    return vault_note.name.startswith('_reference')


def _is_glossary_note(vault_note: VaultNote):
    return vault_note.name.startswith('_glossary')

## PersonalNoteTypeEnum class

In [ ]:
#| export
class PersonalNoteTypeEnum(Enum):
    """
    An Enum class for note types in my Obsidian math vault
    
    """
    INDEX_NOTE = auto()
    STANDARD_INFORMATION_NOTE = auto()
    MODULE_INFORMATION_NOTE = auto()
    INDEX_OF_NOTATION_NOTE = auto()
    NOTATION_NOTE = auto()
    PREMISE_NOTE = auto()
    COMMON_TERMS_NOTE = auto()
    TEMPLATE_NOTE = auto()
    TAG_NOTE = auto()
    NOTES_NOTE = auto()
    MATHEMATICIAN_NOTE = auto()
    EQUATION_NOTE = auto()
    REFERENCE_NOTE = auto()
    GLOSSARY_NOTE = auto()
    
    @classmethod
    def type_function(cls, note_type: 'PersonalNoteTypeEnum'):
        return cls._type_method_dict[note_type]


PersonalNoteTypeEnum._type_method_dict = {
    PersonalNoteTypeEnum.INDEX_NOTE: _is_index_note,
    PersonalNoteTypeEnum.STANDARD_INFORMATION_NOTE: _is_standard_information_note,
    PersonalNoteTypeEnum.MODULE_INFORMATION_NOTE: _is_module_information_note,
    PersonalNoteTypeEnum.INDEX_OF_NOTATION_NOTE: _is_index_of_notation_note,
    PersonalNoteTypeEnum.NOTATION_NOTE: _is_notation_note,
    PersonalNoteTypeEnum.PREMISE_NOTE: _is_premise_note,
    PersonalNoteTypeEnum.COMMON_TERMS_NOTE: _is_common_terms_note,
    PersonalNoteTypeEnum.TEMPLATE_NOTE: _is_template_note,
    PersonalNoteTypeEnum.TAG_NOTE: _is_tag_note,
    PersonalNoteTypeEnum.NOTES_NOTE: _is_notes_note,
    PersonalNoteTypeEnum.MATHEMATICIAN_NOTE: _is_mathematician_note,
    PersonalNoteTypeEnum.EQUATION_NOTE: _is_equation_note,
    PersonalNoteTypeEnum.REFERENCE_NOTE: _is_reference_note,
    PersonalNoteTypeEnum.GLOSSARY_NOTE: _is_glossary_note
}

NameError: name 'Enum' is not defined

In [ ]:
PersonalNoteTypeEnum.type_function(PersonalNoteTypeEnum.INDEX_NOTE)

<function __main__._is_index_note(vault_note: trove.markdown.obsidian.vault.VaultNote) -> bool>

In [ ]:
for note_type in PersonalNoteTypeEnum:
    print(note_type)

PersonalNoteTypeEnum.INDEX_NOTE
PersonalNoteTypeEnum.STANDARD_INFORMATION_NOTE
PersonalNoteTypeEnum.MODULE_INFORMATION_NOTE
PersonalNoteTypeEnum.INDEX_OF_NOTATION_NOTE
PersonalNoteTypeEnum.NOTATION_NOTE
PersonalNoteTypeEnum.PREMISE_NOTE
PersonalNoteTypeEnum.COMMON_TERMS_NOTE
PersonalNoteTypeEnum.TEMPLATE_NOTE
PersonalNoteTypeEnum.TAG_NOTE
PersonalNoteTypeEnum.NOTES_NOTE
PersonalNoteTypeEnum.MATHEMATICIAN_NOTE
PersonalNoteTypeEnum.EQUATION_NOTE
PersonalNoteTypeEnum.REFERENCE_NOTE
PersonalNoteTypeEnum.GLOSSARY_NOTE


In [ ]:
#| export
def type_of_note(vault_note: VaultNote) -> PersonalNoteTypeEnum:
    """Returns the type of the specified note.
    
    **Parameters**
    - vault_note: VaultNote
    
    **Returns**
    - PersonalNoteTypeEnum
    
    **Raises**
    - NoteDoesNoteExistError
        - If `vault_note` does not exist.
    """
    if not vault_note.exists():
        raise NoteDoesNotExistError(vault_note.name)
    for note_type in PersonalNoteTypeEnum:
        type_function = PersonalNoteTypeEnum.type_function(note_type)
        if type_function(vault_note):
            return note_type

In [ ]:
#| export
def note_is_of_type(
        vault_note: VaultNote, note_type: Union[PersonalNoteTypeEnum, None])\
        -> bool:
    """Returns `True` if the Markdown vault note exists and is determined to
    be of the specified type.
    
    **Parameters**
    - vault_note - VaultNote
    - note_type - Union[PersonalNoteTypeEnum, None]
        - The type of note. If `None`, then any type of note;
        in this case, returns whether or not the note exists.
    
    **Returns**
    - `bool`
    
    **Notes**
    - This function may return incorrect results, depending on the 
    formatting of the file.
    """
    if not vault_note.exists():
        return False
    return type_of_note(vault_note) == note_type if note_type else True

We can get the type of a note: